In [2]:
import gurobipy as gp
from gurobipy import GRB
import json
import math
from prettytable import PrettyTable
from typing import List, Mapping, Tuple, Union

In [3]:
def formar_asignaciones(profesores_por_curso: Mapping[str, List[Union[List[str], str]]]) -> List[Tuple[str, str, str]]:
    """Esta función forma la lista de tuplas (curso,profesor,ramo) que deben ser asignadas de forma que puedan ser ingresadas al modelo.

    Args:
        profesores_por_curso (Mapping[str, List[List[str], str]]): Diccionario con cursos como llave. Cada curso contiene una lista que tiene a su vez una lista con el o los profesores que harán ese curso y además el nombre del curso.

    Returns:
        List[Tuple[str, str, str]]: Lista de tuplas de la forma (curso, profesor, ramo) las cuales deben ser asignadas en el horario de cada curso.
    """
    lista_asignaciones = []
    for curso in profesores_por_curso:
        for tupla in profesores_por_curso[curso]:
            for prof in tupla[0]:
                lista_asignaciones.append((curso, prof, tupla[1]))
    return lista_asignaciones

def formar_asignaciones_con_restriccion_modulos_diarios(ramos_por_curso: Mapping[str, Mapping[str, Mapping[str, int]]], asignaciones: List[Tuple[str, str, str]]) -> List[Tuple[str, str, str]]:
    """Esta función forma la lista de las clases que se quiere que necesariamente tengan módulos de clases seguidos.

    Args:
        ramos_con_modulos_seguidos (Mapping[str, Mapping[str, Mapping[str, int]]]): Diccionario con cursos como llave. Cada curso tiene asociado una lista con los ramos que se quiere que tengan módulos dobles. Cada ramo es llave de un diccionario que tiene las llaves "modulos_semanales", "maximo_modulos_diarios" y "tipo_modulos".
        asignaciones (Tuple[str, str, str]): Lista de tuplas de la forma (curso, profesor, ramo) las cuales pueden o deben ser asignadas en el horario de cada curso.

    Returns:
        Tuple[str, str, str]: Lista de tuplas de la forma (curso, profesor, ramo) que deben tener módulos seguidos de clases.
    """ 
    lista_modulos_seguidos = []
    for curso in ramos_por_curso:
        for ramo in ramos_por_curso[curso]:
            if ramos_por_curso[curso][ramo]['tipo_modulos'] == 'seguidos':
                lista_modulos_seguidos.append((curso, ramo))
    asignaciones_con_restriccion_de_modulos_diarios = []
    for elem in asignaciones:
        if (elem[0],elem[2]) in lista_modulos_seguidos:
            asignaciones_con_restriccion_de_modulos_diarios.append(elem)
    return asignaciones_con_restriccion_de_modulos_diarios

def cantidad_maxima_de_dias(carga_profesores: Mapping[str, List[Union[List[str], str]]], ramos_por_curso: Mapping[str, Mapping[str, Mapping[str, int]]]) -> Tuple[Mapping[str, int], List[str]]:
    """Esta función asigna un tope de días de trabajo a los profesores que hacen pocos módulos de clases. En la práctica esto debería ser ingresado por el usuario.

    Args:
        carga_profesores (Mapping[str, List[List[str], str]]): Diccionario con los nombres de los profesores como llave. Cada profesor tiene asociada una lista que contiene una lista de uno o más cursos a los que les hace una cierta clase conjunta. 
        ramos_por_curso (Mapping[str, Mapping[str, Mapping[str, int]]]): Diccionario con los cursos como llave. Cada uno de estos lleva a un diccionario con los ramos como llave, los cuales entregan la cantidad de módulos semanales de cada ramo y el máximo de módulos diarios permitidos.

    Returns:
        Tuple[Mapping[str, int], List[str]]: Tupla que contiene, por una parte, un diccionario con nombres de profesores como llave y entregan el máximo de días que puede trabajar cada uno y, por otra parte, una lista con todos los profesores que no tienen disponibilidad completa.
    """    
    tope_dias = {}
    for profesor in carga_profesores:
        suma = 0
        for tupla in carga_profesores[profesor]:
            suma += ramos_por_curso[tupla[0][0]][tupla[1]]['modulos_semanales']
        if suma <= 2:
            tope_dias[profesor] = 1
        elif suma <= 4:
            tope_dias[profesor] = 2
        elif suma <= 6:
            tope_dias[profesor] = 3
        elif suma <= 8:
            tope_dias[profesor] = 4
    profesores_con_dias_limitados = []
    for elem in tope_dias.keys():
        profesores_con_dias_limitados.append(elem)
    profesores_con_dias_limitados
    
    return tope_dias, profesores_con_dias_limitados

def asignaciones_no_hechas(profesores_a_definir_por_curso: Mapping[str, List[Union[List[str], str]]]) -> List[Tuple[str, str, str]]:
    """Función que forma las tuplas de los cursos que no tienen profesor definido

    Args:
        profesores_a_definir_por_curso (Mapping[str, List[List[str], str]]): Diccionario con cursos como llave. Cada curso contiene una lista que tiene a su vez una lista con el o los profesores que pueden hacer ese curso y además el nombre del curso.

    Returns:
        List[Tuple[str, str, str]]: Lista de tuplas de la forma (curso, profesor, ramo) las cuales pueden ser asignadas en el horario de cada curso.
    """    
    no_asignaciones = []
    for curso in profesores_a_definir_por_curso:
        for tupla in profesores_a_definir_por_curso[curso]:
            for prof in tupla[0]:
                no_asignaciones.append((curso, prof, tupla[1]))
    return no_asignaciones

In [4]:
model = gp.Model("Generación de horarios de colegio")
# model.setParam('OutputFlag', 1)
f = open('probando_formato.json',encoding='utf8')

M = 10000
data = json.load(f)
dias = data['dias']
modulos = data['modulos']
horarios = data['horarios']
profesores_por_curso = data['profesores_por_curso']
ramos_por_curso = data['ramos_por_curso']
carga_profesores = data['carga_profesores']
modulos_consecutivos = data['modulos_consecutivos']
limitaciones_profesores = data['limitaciones_profesores']
profesores_a_definir_por_curso = data['profesores_a_definir_por_curso']
asignaciones_fijadas = data['asignaciones_fijadas']
asignaciones_vetadas = data['asignaciones_vetadas']
ramos_en_distintos_dias = data['ramos_en_distintos_dias']

asignaciones_normales = formar_asignaciones(profesores_por_curso)
no_asignaciones = []
# no_asignaciones = asignaciones_no_hechas(profesores_por_curso)
total_asignaciones = asignaciones_normales + no_asignaciones
asignaciones_con_restriccion_de_modulos_diarios = formar_asignaciones_con_restriccion_modulos_diarios(
    ramos_por_curso, total_asignaciones
)
# asignaciones_multiples, cantidad_de_cursos_simultaneos = formar_asignaciones_multiples(multiples_cursos)
tope_dias, profesores_con_dias_limitados = cantidad_maxima_de_dias(carga_profesores, ramos_por_curso)


x = model.addVars(total_asignaciones,dias,modulos,vtype=GRB.BINARY, name="x")
s = model.addVars(asignaciones_con_restriccion_de_modulos_diarios,dias,vtype=GRB.BINARY, name="s")
# * Solo para los cursos donde se prefieren módulos dobles
w = model.addVars(no_asignaciones, vtype=GRB.BINARY, name="w")
y = model.addVars(profesores_con_dias_limitados,dias,vtype=GRB.BINARY,name="y") # * Solo para los profesores que tienen menos de 5 días

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-17


In [5]:
model.addConstrs((sum(x[c,ps[0],r,d,j] for ps,r in profesores_por_curso[c]) + sum(x[c,p,r,d,j] for ps,r in profesores_a_definir_por_curso[c] for p in ps) <= 1 for c in horarios for d in horarios[c] for j in horarios[c][d]),name="R1");

In [6]:
model.addConstrs((sum(x[c,p,r,d,j] for ps,r in profesores_por_curso[c] for p in ps) + sum(x[c,p,r,d,j] for ps,r in profesores_a_definir_por_curso[c] for p in ps) >= 1 for c in horarios for d in horarios[c] for j in horarios[c][d]),name="R2");

In [7]:
for c in profesores_por_curso:
    for ps,r in profesores_por_curso[c]:
        if len(ps) > 1:
            model.addConstrs((x[c,p1,r,d,j] - x[c,p2,r,d,j] == 0 for p1,p2 in [(ps[a],ps[b]) for a in range(len(ps)) for b in range(len(ps)) if a < b] for d in dias for j in modulos), name=f"R3[{c},{r}]");

# ¡¡Restricciones sin profesores faltan acá!!

In [8]:
for c in profesores_por_curso:
    for ps,r in profesores_por_curso[c]:
        model.addConstrs((sum(x[c,p,r,d,j] for d in dias for j in modulos) == ramos_por_curso[c][r]['modulos_semanales'] for p in ps),name=f"R4[{c},{r}]");

In [9]:
for c in profesores_a_definir_por_curso:
    for ps,r in profesores_a_definir_por_curso[c]:
        aux = []
        for p in ps:
            aux.append(p)
        aux = tuple(aux)
        model.addConstrs((sum(x[c,q,r,d,j] for q in aux for d in dias for j in modulos) == ramos_por_curso[c][r]['modulos_semanales']),name=f"R5[{c},{r}]")


In [10]:
model.addConstrs((sum(x[c,p,r,d,j] for ps,r in profesores_por_curso[c] for p in ps) + sum(x[c,p,r,d,j] for ps,r in profesores_a_definir_por_curso[c] for p in ps) == 0 for c in horarios for d in horarios[c] for j in list(set(modulos) - set(horarios[c][d]))),name="R6");

In [11]:
model.addConstrs((sum(x[cs[0],p,r,d,j] for cs,r in carga_profesores[p]) <= 1 for p in carga_profesores for d in dias for j in modulos),name="R7");

In [12]:
for p in carga_profesores:
    for cs,r in carga_profesores[p]:
        model.addConstrs((x[c1,p,r,d,j] - x[c2,p,r,d,j] == 0 for c1,c2 in [(cs[a],cs[b]) for a in range(len(cs)) for b in range(len(cs)) if a < b] for d in dias for j in modulos), name=f"R8[{p},{r}]")

In [13]:
for c in profesores_por_curso:
    for ps,r in profesores_por_curso[c]:
        model.addConstrs((sum(x[c,ps[0],r,d,j] for j in modulos) <= ramos_por_curso[c][r]['maximo_modulos_diarios'] for d in dias),name=f"R9[{c},{p},{r}]");

In [14]:
for c in profesores_a_definir_por_curso:
    for ps,r in profesores_a_definir_por_curso[c]:
        aux = []
        for p in ps:
            aux.append(p)
        aux = tuple(aux)
        model.addConstrs((sum(x[c,q,r,d,j] for q in aux for j in modulos) <= ramos_por_curso[c][r]['maximo_modulos_diarios'] for d in dias),name=f"R10[{c},{r}]")

In [15]:
model.addConstrs((x[c,p,r,d,j] + x[c,p,r,d,z] <= 1 for c,p,r in total_asignaciones if ramos_por_curso[c][r]['tipo_modulos'] == 'seguidos' for d in dias for j,z in list(set([(a,b) for a in horarios[c][d] for b in horarios[c][d] if a < b]) - set([tuple(comb) for comb in modulos_consecutivos]))), name="R11");

In [16]:
model.addConstrs((x[c,p,r,d,j] + x[c,p,r,d,z] <= 1 for c,p,r in total_asignaciones if ramos_por_curso[c][r]['tipo_modulos'] == 'disjuntos' for d in dias for j,z in modulos_consecutivos), name="R12");

In [26]:
model.addConstrs((M * s[c,p,r,d] >= sum(x[c,p,r,d,j] for j in modulos) for c,p,r in asignaciones_con_restriccion_de_modulos_diarios for d in dias),name="R13");

In [27]:
model.addConstrs((sum(s[c,p,r,d] for d in dias) <= math.ceil(ramos_por_curso[c][r]['modulos_semanales'] / 2) for c,p,r in asignaciones_con_restriccion_de_modulos_diarios),name="R14");

In [17]:
model.addConstrs((sum(x[cs[0],p,r,d,j] for cs,r in carga_profesores[p] for d in dias for j in modulos) <= limitaciones_profesores[p]['maximo_modulos'] for p in carga_profesores),name="R15");

In [18]:
model.addConstrs((sum(x[cs[0],p,r,d,j] for cs,r in carga_profesores[p] for j in modulos) <= limitaciones_profesores[p]['maximo_modulos_diario'] for p in carga_profesores for d in dias),name="R16");

In [19]:
model.addConstrs((sum(x[cs[0],p,r,d,j1] + x[cs[0],p,r,d,j2] + x[cs[0],p,r,d,j3] + x[cs[0],p,r,d,j4] + x[cs[0],p,r,d,j5] for cs,r in carga_profesores[p]) <= 4 for p in carga_profesores for d in dias for j1,j2,j3,j4,j5 in [(i,i+1,i+2,i+3,i+4) for i in modulos if i+4 in modulos]), name="R17");

In [401]:
model.addConstrs((M * y[p,d] >= sum(x[cs[0],p,r,d,j] for cs,r in carga_profesores[p] for j in modulos) for d in dias for p in profesores_con_dias_limitados),name="R18");

In [402]:
model.addConstrs((sum(y[p,d] for d in dias) <= tope_dias[p] for p in profesores_con_dias_limitados),name="R19");

In [20]:
for c in profesores_a_definir_por_curso:
    for ps,r in profesores_a_definir_por_curso[c]:
        model.addConstrs((M * w[c,p,r] >= sum(x[c,p,r,d,j] for d in horarios[c] for j in horarios[c][d]) for p in ps),name=f"R20[{c},{r}]");

In [21]:
model.addConstrs((sum(w[c,p,r] for ps,r in profesores_a_definir_por_curso[c] for p in ps) <= 1 for c in profesores_a_definir_por_curso),name="R21");

In [22]:
model.addConstrs((x[c,p,r,d,j] == 1 for c,p,r,d,j in asignaciones_fijadas),name="R22");

lista_restricciones = []
for c,p,r,d,j in asignaciones_fijadas:
    lista_restricciones.append(f"R22[{c},{p},{r},{d},{j}]")

In [23]:
model.addConstrs((x[c,p,r,d,j] == 0 for c,p,r,d,j in asignaciones_vetadas),name="R23");

for c,p,r,d,j in asignaciones_vetadas:
    lista_restricciones.append(f"R23[{c},{p},{r},{d},{j}]")

In [24]:
for c in ramos_en_distintos_dias:
    for ls in ramos_en_distintos_dias[c]:
        for p1,r1,p2,r2 in [(ls[b][0][i],ls[b][1],ls[d][0][k],ls[d][1]) for b in range(len(ls)) for d in range(len(ls)) if b < d for i in range(len(ls[b][0])) for k in range(len(ls[d][0]))]:
                model.addConstrs((sum(x[c,p1,r1,d,j] +  x[c,p2,r2,d,j] for j in modulos) <= max(ramos_por_curso[c][r1]['maximo_modulos_diarios'], ramos_por_curso[c][r2]['maximo_modulos_diarios']) for d in dias),name="R24");

In [28]:
obj = sum(x[c,p,r,d,j] + x[c2,p2,r2,d,z] for c,p,r in asignaciones_normales for c2,p2,r2 in asignaciones_normales if p == p2 and c == c2 for d in dias for j,z in modulos_consecutivos)
model.setObjective(0, GRB.MINIMIZE)

model.setParam("PoolSolutions", 10)
model.setParam('PoolSearchMode', 2)
model.setParam('OutputFlag', 1)
model.update()

seguir = True
while seguir:
    model.optimize()
    if 'value' in str(model.getVars()[0]):
        seguir = False
    if seguir:
        model.remove(model.getConstrByName(lista_restricciones[len(lista_restricciones) - 1]))
        lista_restricciones.remove(lista_restricciones[len(lista_restricciones) - 1])
        model.update()
        
# model.computeIIS()
# removed =[]
# for c in model.getConstrs():
#     if c.IISConstr:
#         print('%s' % c.constrName)
#         # Remove a single constraint from the model
#         removed.append(str(c.constrName))
#         model.remove(c)

model.update()
model.write('probando.lp')
for i in range(model.SolCount):
    model.setParam("SolutionNumber", i)
    model.update()

    print(model.optimize())
    model.write(f"solutions/out{i+1}.sol")


# model.optimize()
# model.write('out.sol')

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 59688 rows, 18190 columns and 270242 nonzeros
Model fingerprint: 0x6183cf7e
Variable types: 0 continuous, 18190 integer (18190 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint R13[1A,Camila_Sutar,Matematica,Lunes] by 2.000000000

Presolve removed 48202 rows and 6478 columns
Presolve time: 0.78s
Presolved: 11486 rows, 11712 columns, 113057 nonzeros
Variable types: 0 continuous, 11712 integer (11712 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   26846    0.000000

In [33]:
def leer_output(numero_sol):
    lista_final = []
    with open(f'solutions/out{numero_sol}.sol', 'r') as file:
        lines = file.readlines()[2:]
        for line in lines:
            if line[-2] == '1':
                if line[-4] == ']':
                    line = line.strip('\n')
                    lista_final.append(line)
                else:
                    line = line.strip('\n')
                    if round(float(line[line.find(']') + 2:])) == 1:
                        lista_final.append(line[:line.find(']') + 1] + ' 1')
            else:
                if line[-4] != ']':
                    line = line.strip('\n')
                    if int(float(line[line.find(']') + 2:])) == 1:
                        lista_final.append(line[:line.find(']') + 1] + ' 1')

    diccionario_dias = {
        'Lunes': 1,
        'Martes': 2,
        'Miercoles': 3,
        'Jueves': 4,
        'Viernes': 5
    }

    vuelta = {
        1: 'Lunes',
        2: 'Martes',
        3: 'Miercoles',
        4: 'Jueves',
        5: 'Viernes'
    }

    profesores_final = []
    for elem in lista_final:
        if elem[0] == "x":
            elem = elem[2:-3]
            elem = elem.split(',')
            profesores_final.append(elem)
            
    for i in range(len(profesores_final)):
        profesores_final[i][3] = diccionario_dias[profesores_final[i][3]]
    profesores_final = sorted(profesores_final, key= lambda x: int(x[4]))
    profesores_final = sorted(profesores_final, key= lambda x: x[3])
    profesores_final = sorted(profesores_final, key= lambda x: x[0])

    for i in range(len(profesores_final)):
        profesores_final[i][3] = vuelta[profesores_final[i][3]]

    ultimo_curso = None
    ultimo_ramo = None
    ultimo_dia = None
    ultimo_modulo = None
    indices = []
    for i in range(len(profesores_final)):
        if profesores_final[i][0] == ultimo_curso:
            if profesores_final[i][2] == ultimo_ramo:
                if profesores_final[i][3] == ultimo_dia:
                    if profesores_final[i][4] == ultimo_modulo:
                        indices.append(i)
        ultimo_curso = profesores_final[i][0]
        ultimo_ramo = profesores_final[i][2]
        ultimo_dia = profesores_final[i][3]
        ultimo_modulo = profesores_final[i][4]
    indices.reverse()
    for indice in indices:
        profesores_final[indice - 1][1] = profesores_final[indice - 1][1] + ' & ' + profesores_final[indice][1]
        profesores_final.pop(indice)
    return profesores_final

def generar_horario(curso, numero_sol):

    clases_curso = []
    dias_semana = []
    ultimo_dia = None
    horario = PrettyTable(dias_semana)
    lista_auxiliar = []
    lista_dia = None
    for elem in leer_output(numero_sol):    
        if elem[0] == curso:
            clases_curso.append([elem[2], elem[3], elem[4], elem[1]])
            if elem[3] not in dias_semana:
                dias_semana.append(elem[3])
            if elem[3] != ultimo_dia:
                lista_auxiliar.append(lista_dia)
                lista_dia = []
                ultimo_dia = elem[3]
                lista_dia.append(ultimo_dia)
                elem[2] = elem[2].replace('_', ' ')
                elem[1] = elem[1].replace('_', ' ')
                lista_dia.append(f'{elem[2]} \n {elem[1]}')
            else:
                elem[2] = elem[2].replace('_', ' ')
                elem[1] = elem[1].replace('_', ' ')
                lista_dia.append(f'{elem[2]} \n {elem[1]}')

    lista_auxiliar.append(lista_dia)
    lista_auxiliar = lista_auxiliar[1:]
    len_maxima = 0
    for dia in lista_auxiliar:
        if len(dia) > len_maxima:
            len_maxima = len(dia)

    horario.add_column('', [i+1 for i in range(len_maxima - 1)])
    
    for i in range(len(lista_auxiliar)):
        for j in range(len_maxima - len(lista_auxiliar[i])):
            lista_auxiliar[i].append('')
        horario.add_column(lista_auxiliar[i][0], lista_auxiliar[i][1:], align='c')
    horario.format = True   
    print(f'Horario del curso {curso}: ')   
    return horario

curso = 'IVA'
generar_horario(curso,numero_sol=1)

Horario del curso IVA: 


,Lunes,Martes,Miercoles,Jueves,Viernes
1,Quimica Comun Beatriz Arancibia,Biologia Comun/Matematica M2-2 Marcela Bravo & Andres Castillo,Fisica Comun Carlos Claros,Competencia Lectora Marcela Arias,Educacion Fisica y Salud Guillermo X & Joaquina X
2,Quimica Comun Beatriz Arancibia,Biologia Comun/Matematica M2-2 Marcela Bravo & Andres Castillo,Fisica Comun Carlos Claros,Competencia Lectora Marcela Arias,Educacion Fisica y Salud Guillermo X & Joaquina X
3,Ensayo3 Marcela Arias,Ensayo1 Maricel X & Patricio X,Competencia Lectora Marcela Arias,Electivo Marcela Bravo & Carlos Claros & Beatriz Arancibia,Matematica M1-1 Eraldo Cortes
4,Ensayo2 Andrea Perez,Ensayo1 Maricel X & Patricio X,Competencia Lectora Marcela Arias,Electivo Marcela Bravo & Carlos Claros & Beatriz Arancibia,Matematica M1-1 Eraldo Cortes
5,Biologia Comun/Matematica M2-1 Marcela Bravo & Carlos Claros,Fisica Comun Carlos Claros,Electivo Marcela Bravo & Carlos Claros & Beatriz Arancibia,Ensayo4 Paula Ceballos,Historia y Ciencias Sociales Luis Reyes
6,Biologia Comun/Matematica M2-1 Marcela Bravo & Carlos Claros,Fisica Comun Carlos Claros,Electivo Marcela Bravo & Carlos Claros & Beatriz Arancibia,Ensayo4 Paula Ceballos,Historia y Ciencias Sociales Luis Reyes
7,Competencia Lectora Marcela Arias,Matematica M1-2 Andres Castillo,Quimica Comun Beatriz Arancibia,Historia y Ciencias Sociales Luis Reyes,
8,Competencia Lectora Marcela Arias,Matematica M1-2 Andres Castillo,Quimica Comun Beatriz Arancibia,Historia y Ciencias Sociales Luis Reyes,
9,Matematica M1-1 Eraldo Cortes,Historia y Ciencias Sociales Luis Reyes,Ensayo2 Andrea Perez,,
10,Matematica M1-1 Eraldo Cortes,Historia y Ciencias Sociales Luis Reyes,Ensayo2 Andrea Perez,,


In [32]:
def generar_horario_profesor(nombre_profesor,numero_sol):
    clases_profesor = []
    dict_dias = {'Lunes': 1, 'Martes': 2, 'Miercoles': 3, 'Jueves': 4, 'Viernes': 5, 'Sábado': 6, 'Domingo': 7}
    dict_dias_inverso = {1: 'Lunes', 2: 'Martes', 3: 'Miercoles', 4: 'Jueves', 5: 'Viernes', 6: 'Sábado', 7: 'Domingo'}
    for elem in leer_output(numero_sol):
        if elem[1] == nombre_profesor:
            clases_profesor.append(elem)
        elif '&' in elem[1]:
            if nombre_profesor in elem[1]:
                clases_profesor.append([elem[0], nombre_profesor, elem[2], elem[3], elem[4]])
            
    for i in range(len(clases_profesor)):
        clases_profesor[i][3] = dict_dias[clases_profesor[i][3]]
    clases_profesor = sorted(clases_profesor, key= lambda x: int(x[4]))
    clases_profesor = sorted(clases_profesor, key= lambda x: x[3])
    for i in range(len(clases_profesor)):
        clases_profesor[i][3] = dict_dias_inverso[clases_profesor[i][3]]

    ingreso_clase = False
    lista_auxiliar = []
    for i in dias:
        lista_dia = [i]
        for j in modulos:
            if clases_profesor == []:
                lista_dia.append(' ')
            elif clases_profesor != []:
                ingreso_clase = False
                primera = True
                if i != clases_profesor[0][3] or j != int(clases_profesor[0][4]):
                    lista_dia.append(' ')          
                while i == clases_profesor[0][3] and j == int(clases_profesor[0][4]):
                    ingreso_clase = True
                    if primera:
                        cursos = f'{clases_profesor[0][0]}'
                    else:
                        cursos += f' & {clases_profesor[0][0]}'
                    ramo = clases_profesor[0][2]
                    clases_profesor.pop(0)
                    primera = False
                    if clases_profesor == []:
                        break
                if ingreso_clase:
                    lista_dia.append(f'{cursos} \n {ramo}')
        lista_auxiliar.append(lista_dia)

    # print(dias)
    horario = PrettyTable([])
    horario.add_column('', modulos)
    # print(modulos)
    # print(horario)
    for i in range(len(dias)):
        # print(lista_auxiliar)
        horario.add_column(lista_auxiliar[i][0], lista_auxiliar[i][1:], align='c')
    horario.format = True   
    print(f'Horario del profesor {nombre_profesor}: ')   
    return horario

generar_horario_profesor(nombre_profesor='Joaquina_X', numero_sol=7)

Horario del profesor Joaquina_X: 


,Lunes,Martes,Miercoles,Jueves,Viernes
1,,4A & 4B Educacion_Fisica_y_Salud,6A & 6B Educacion_Fisica_y_Salud,,
2,,4A & 4B Educacion_Fisica_y_Salud,6A & 6B Educacion_Fisica_y_Salud,,
3,7A & 7B Educacion_Fisica_y_Salud,7A & 7B Educacion_Fisica_y_Salud,,,6A & 6B Educacion_Fisica_y_Salud
4,7A & 7B Educacion_Fisica_y_Salud,7A & 7B Educacion_Fisica_y_Salud,,,6A & 6B Educacion_Fisica_y_Salud
5,,,5A & 5B Educacion_Fisica_y_Salud,5A & 5B Educacion_Fisica_y_Salud,4A & 4B Educacion_Fisica_y_Salud
6,,,5A & 5B Educacion_Fisica_y_Salud,5A & 5B Educacion_Fisica_y_Salud,4A & 4B Educacion_Fisica_y_Salud
7,,,,,
8,,,,,
9,,IVA & IVB Educacion_Fisica_y_Salud,,,
10,,IVA & IVB Educacion_Fisica_y_Salud,,,
